# Priority Places for Food - Data Preparation

This notebook prepares the data required for constructing the Priority Places for Food Index. It should run successfully using only the data and code contained within the repository. The notebook creates a raw data directory, which is used to house data downloaded from the web, and a processed data directory. The repo also contains a .devcontainer which can be used with VSCode and Docker to create the environment to run the below notebook. The default paths for the data directories assume that this is how the notebook is running.

## Setup

In [ ]:
# Install required packages
!pip install geovoronoi
!pip install openpyxl==3.1.0
!pip install odfpy
!pip install xmltodict
!pip install xlrd

In [ ]:
# Import required packages
import geopandas as gpd
import pandas as pd
import operator
import sys
import os
from urllib.request import urlopen
from shutil import copyfileobj, unpack_archive
import openpyxl
import requests
import json
import xml.etree.ElementTree as ET
import xmltodict
import io
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'DistanceCalculator'))
from bipartite_nearest_distance_calculator import calculate_bipartite_nearest_distance, count_bipartite_within_tolerance

In [ ]:
# Set and make data directories
raw_data_directory = '/workspaces/priority-places-calculator/data/raw/'
processed_data_directory = '/workspaces/priority-places-calculator/data/processed/'

os.makedirs(raw_data_directory, exist_ok=True)
os.makedirs(processed_data_directory, exist_ok=True)

## Utility Functions

In [ ]:
def get_raw_data(url, filepath, compressed_file=False, uncompressed_filepath=None):
    '''
    This utility function checks to see if filepath exists and if it doesn't, then it will create the file from the provided url.
    
    There is additional functionality for compressed files
    
    Returns None - the resulting file still needs to loaded.
    '''
    if not os.path.exists(filepath):
        with urlopen(url) as in_stream, open(filepath, 'wb') as out_file:
            copyfileobj(in_stream, out_file)
    if compressed_file and not os.path.exists(uncompressed_filepath):
        unpack_archive(filepath, uncompressed_filepath)
    return


def get_country_outline():
    ''' 
    Downloads a country outline if country_outline variable is not populated.
    
    Returns country_outline
    ''' 
    if 'country_outline' not in globals():
        country_outline = gpd.read_file('https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Countries_December_2021_UK_BGC_2022/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
        country_outline = country_outline.to_crs('EPSG:4326')
        country_outline = country_outline.dissolve()

    return country_outline


def load_postcode_data():
    '''
    Loads postcode data from the geoportal. Loads if postcodes object doesn't already exist.
    '''
    get_raw_data(url='https://geoportal.statistics.gov.uk/datasets/0d58125f0f9f4c81b6370a10f5ea3309_0.csv', 
                 filepath=raw_data_directory + 'postcodes.csv')
    if 'postcodes' not in globals():
        postcodes_local = pd.read_csv(raw_data_directory + 'postcodes.csv', usecols=['PCD', 'LSOA11', 'OSLAUA', 'X', 'Y'])
        postcodes_local = gpd.GeoDataFrame(postcodes, geometry=gpd.points_from_xy(postcodes['X'], postcodes['Y']), crs=4326)
        return postcodes_local
    else:
        return postcodes



def load_fsa_data(postcodes):
    ''' 
    Scrapes, loads and cleans the FSA data
    '''
    
    
    if not os.path.exists(raw_data_directory + 'FSA_establishment_data.csv'):
        x = requests.get('https://api.ratings.food.gov.uk/Authorities', headers={'x-api-version':'2'})
        authorities = json.loads(x.content)
        open_data_files = [authorities['authorities'][i]['FileName'] for i in range(len(authorities['authorities']))]
        fsa = pd.DataFrame()
        for url in open_data_files:
            print(url)
            response = requests.get(url)
            xml_data = ET.fromstring(response.content)
            xmlstr = ET.tostring(xml_data, encoding='utf-8', method='xml')
            data_dict = xmltodict.parse(xmlstr)
            establishments = data_dict['FHRSEstablishment']['EstablishmentCollection']['EstablishmentDetail']
            df = pd.json_normalize(establishments)
            fsa = pd.concat([fsa, df], axis=0)
        fsa.to_csv(raw_data_directory + 'FSA_establishment_data.csv', index=False)
    else:
        fsa = pd.read_csv(raw_data_directory + 'FSA_establishment_data.csv')

    # Relating to 'Proximity to and density of Non-supermarket food provision', restrict the data to 'Retailers - other'
    fsa = fsa[fsa['BusinessType'].isin(['Retailers - other'])]

    # The largest proportion of geographic data is avialable at the postcode level. 
    # In an effort to clean up the location data, let's take any data with a missing long/lat field and see if we can map the postcode to lon/lat positions.
    print('Total records: %d' % fsa.shape[0])
    print('Total records without a geocode: %d' % fsa['Geocode.Latitude'].isna().sum())

    joined = fsa.merge(postcodes[['PCD', 'X', 'Y', 'geometry']], left_on='PostCode', right_on='PCD', how='left')
    joined['Geocode.Latitude'] = joined['Geocode.Latitude'].fillna(joined.Y)
    joined['Geocode.Longitude'] = joined['Geocode.Longitude'].fillna(joined.X)
    fsa = joined[fsa.columns]

    print('Total records without a geocode: %d' % fsa['Geocode.Latitude'].isna().sum())

    # Filter out records without a geocode
    fsa = fsa[~fsa['Geocode.Latitude'].isna()]

    fsa = gpd.GeoDataFrame(fsa, geometry=gpd.points_from_xy(fsa['Geocode.Longitude'], fsa['Geocode.Latitude']), crs='EPSG:4326')

    return fsa


## Proximity to supermarket retail facilities

### Average distance to nearest large grocery store

For England, Scotland and Wales, indicator data obtained from e-food desert index of Newing et al., 2020. Location data of large grocery stores obtained from Geolytix Retails Points v15 and distances computed using GIS software. Code for Northern Ireland calculation shown below.

In [ ]:
get_raw_data(url='https://drive.google.com/uc?id=1B8M7m86rQg2sx2TsHhFa2d-x-dZ1DbSy', 
             filepath=raw_data_directory+'geolytix.zip',
             compressed_file=True, 
             uncompressed_filepath=raw_data_directory+'geolytix')

supermarkets = pd.read_csv(raw_data_directory + 'geolytix/Previous Versions/geolytix_retailpoints_v15_202001.csv')
supermarkets = gpd.GeoDataFrame(supermarkets, geometry=gpd.points_from_xy(supermarkets['long_wgs'], supermarkets['lat_wgs']), crs=4326)

large_supermarkets = supermarkets[supermarkets.size_band.isin(['15,069 < 30,138 ft2 (1,400 < 2,800 m2)', '30,138 ft2 > (2,800 m2)'])]
large_supermarkets = gpd.GeoDataFrame(large_supermarkets, geometry=gpd.points_from_xy(large_supermarkets['long_wgs'], large_supermarkets['lat_wgs']), crs=4326)

country_outline = get_country_outline()

large_supermarkets = gpd.sjoin(large_supermarkets, country_outline, how="inner", predicate='intersects')

In [ ]:
postcodes = load_postcode_data()

pntsB = large_supermarkets[['id', 'geometry']].to_crs(4326)
pntsB.set_index('id', inplace=True)

pntsA = postcodes[['PCD', 'geometry']].to_crs(4326)
pntsA.set_index('PCD', inplace=True)

In [ ]:
# Do the distance calculation
Polygon = country_outline.envelope.values[0]

dist = calculate_bipartite_nearest_distance(pntsA, pntsB, Polygon)
dist.to_csv(processed_data_directory + 'postcode_to_nearest_large_supermarket_distance.csv', index=True)

### Average count of stores within 1km

For England, Scotland and Wales, indicator data obtained from e-food desert index of Newing et al., 2020. Location data of large grocery stores obtained from Geolytix Retails Points v15 and distances computed using GIS software. Code for Northern Ireland shown below. 

In [ ]:
# Uses variables from the above calculation

tolerance = 1000

pntsA = postcodes[['PCD', 'geometry']]
supermarket_within_1km = count_bipartite_within_tolerance(pntsA, pntsB, 1000)
supermarket_within_1km = pd.DataFrame(supermarket_within_1km).merge(pntsA, left_index=True, right_index=True, how='inner')
supermarket_within_1km.to_csv(processed_data_directory + 'postcode_to_nearest_large_supermarket_1km_count.csv', index=True)

## Accessibility to supermarket retail facilities

### Average travel distance (based on a custom built spatial interaction model)

For England, Scotland and Wales, values are taken from the same indicator in the e-food desert index of Newing et al., 2020. These are provided within the repository and used directly for index construction.

### Accessibility via public transport (Govt Journey Time Statistics 2017 - 2020)

For England, Scotland and Wales, values are taken from the same indicator in the e-food desert index of Newing et al., 2020. These are provided within the repository and used directly for index construction.

## Access to online deliveries

### Online groceries availability (Newing et al, 2020). E,S,W

For England, Scotland and Wales, vaues are taken from the same indicator in the e-food desert index of Newing et al., 2020. These are provided within the repository and used directly for index construction.

### Propensity to shop online (CDRC Internet User Classification 2018). E,S,W.

In [ ]:
# Download the following files:
# 1. IUC 2018 Cluster Centres
# 2. IUC 2018 (CSV)
# from cdrc: 
# https://data.cdrc.ac.uk/dataset/internet-user-classification
if not os.path.exists(raw_data_directory + 'iuc2018clustercentres.csv'):
    raise FileNotFoundError("Internet user classification data not found in raw data directory. Please download from: https://data.cdrc.ac.uk/dataset/internet-user-classification")

iuc_centres = pd.read_csv(raw_data_directory + 'iuc2018clustercentres.csv', usecols=['Cluster Hierarchy', 'Cluster Group', 'Online Shopping: Buy Groceries'])
iuc2018 = pd.read_csv(raw_data_directory + 'iuc2018.csv')
shoponline = iuc2018.merge(iuc_centres, left_on='GRP_CD', right_on='Cluster Hierarchy', how='left')
shoponline.rename({'LSOA11_CD':'LSOA11CD', 'Online Shopping: Buy Groceries': 'zshoponline'}, inplace=True, axis=1)
shoponline[['LSOA11CD', 'zshoponline']].to_csv(processed_data_directory + 'propensity_shop_online.csv', index=False)

## Proximity to non-supermarket food provision

In [ ]:
postcodes = load_postcode_data()
fsa = load_fsa_data(postcodes)

### Distance to nearest non-supermarket retail food store (Food Standards Agency, accessed 2022-08-23). E,S,W,NI

In [ ]:
for region in postcodes[~postcodes.OSLAUA.isna()].OSLAUA.str[0:3].unique():
    pcds_rgn = postcodes[postcodes['OSLAUA'].str[0:3] == region]
    pntsA = pcds_rgn[['PCD', 'geometry']]
    pntsA.set_index('PCD', inplace=True)
    pntsB = fsa[['FHRSID', 'geometry']]
    polygon = country_outline.envelope.values[0]
    dist = calculate_bipartite_nearest_distance(pntsA, pntsB, polygon)
    dist.to_csv(processed_data_directory + region + '_pcd_nonsupermarket_dist.csv')

### Count of non-supermarket retail food stores within 1km (Food Standards Agency, accessed 2022-08-23). E,S,W,NI

In [ ]:
# Relies on fsa data being loaded in the previous cell

# Convert FSA data to UK BNG
pntsB = fsa[['FHRSID', 'geometry']].to_crs(27700)

# Loop through regions to batch things up a bit
for region in postcodes[~postcodes.OSLAUA.isna()].OSLAUA.str[0:3].unique():

    print('Region currently processing: %s' % region)
    # Initialise regional count
    pcd_intersect_counts = pd.DataFrame()

    # Filter on region 
    pcds_rgn = postcodes[postcodes['OSLAUA'].str[0:3] == region]

    # Reproject to UK BNG
    pcds_rgn = pcds_rgn.to_crs(27700)

    # Calculate the postcode buffer
    pcds_rgn.geometry = pcds_rgn.buffer(1000)

    print('Number of PCDs in region: %d' % (pcds_rgn.shape[0]))

    # Batch the analysis
    batchsize = int(pcds_rgn.shape[0] / 10.0)
    row = 0
    batch_counter = 0
    while row < pcds_rgn.shape[0]:
        print("Batch %d -- Processing pcds_rgn rows %d to %d" % (batch_counter, row, row+batchsize))

        # Join and group by in a single step to prevent any large dfs hanging around
        pcd_intersect_counts_batch = gpd.sjoin(pcds_rgn[row:row+batchsize], pntsB, how='left').groupby('PCD')['FHRSID'].count()

        # Save the batch
        pcd_intersect_counts = pd.concat([pcd_intersect_counts, pcd_intersect_counts_batch])
        batch_counter += 1
        row += batchsize

    pcd_intersect_counts.to_csv(processed_data_directory + region + '_pcd_nonsupermarket_1kmcount.csv')


### Average distance to nearest market (CDRC data from National Market Traders Federation 2016-2019). E,W.

In [ ]:
# Data can be downloaded from https://data.cdrc.ac.uk/dataset/national-market-traders-federation
# and should be saved in the raw_data directory.

if not os.path.exists(raw_data_directory + 'nmftmarkets.xlsx'):
    raise FileNotFoundError("National market traders federation data not found in raw data directory. Please download from: https://data.cdrc.ac.uk/dataset/national-market-traders-federation")

nmftmarkets = pd.read_excel(raw_data_directory + 'nmftmarkets.xlsx')
nmftmarkets = gpd.GeoDataFrame(nmftmarkets, geometry=gpd.points_from_xy(nmftmarkets.longitude, nmftmarkets.latitude), crs=4326)
nmftmarkets = nmftmarkets[nmftmarkets['type_specific_mkv'].isin(['Retail', 'Food and Drink', 'Food and Drink/Arts and Crafts', 'General', 'Speciality Food and Drink', 'Food'])]

postcodes=load_postcode_data()

pntsA = postcodes[['PCD', 'geometry']]
pntsA.set_index('PCD', inplace=True)
pntsB = nmftmarkets[['name_mkv', 'geometry']]
polygon = country_outline.envelope.values[0]
dist = calculate_bipartite_nearest_distance(pntsA, pntsB, polygon)
dist.to_csv(processed_data_directory + 'pcd_nmftmarkets_dist.csv')

### Average count of markets within 1km (CDRC data from National Market Traders Federation 2016-2019). E,W.

In [ ]:
# Relies on data from the previous cell
market_within_1km = count_bipartite_within_tolerance(pntsA, pntsB, 1000)
market_within_1km.to_csv(processed_data_directory + 'postcode_market_1km_count.csv', index=True)

## Socio-demographic barriers

### Proportion of population experiencing income deprivation (UK Govt Index of Multiple Deprivation 2017-2020). E,S,W,NI. 

In [ ]:
# England
population_denominators_url = "https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833980/File_6_-_IoD2019_Population_Denominators.xlsx"
population_denominators_filepath = raw_data_directory + 'File_6_-_IoD2019_Population_Denominators.xlsx'
get_raw_data(population_denominators_url, population_denominators_filepath)
xls_denominators = pd.ExcelFile(population_denominators_filepath)
population_denominators = pd.read_excel(xls_denominators, sheet_name=1)

underlying_indicators_url = "https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833992/File_8_-_IoD2019_Underlying_Indicators.xlsx"
underlying_indicators_filepath = raw_data_directory + 'File_8_-_IoD2019_Underlying_Indicators.xlsx'
get_raw_data(underlying_indicators_url, underlying_indicators_filepath)
xls_indicators = pd.ExcelFile(underlying_indicators_filepath)
underlying_indicators = pd.read_excel(xls_indicators, sheet_name=1)

inc_dep = population_denominators.merge(underlying_indicators, on="LSOA code (2011)", how='inner')
inc_dep = inc_dep[['LSOA code (2011)', 'Total population: mid 2015 (excluding prisoners)', 'Income Domain numerator']]
inc_dep['Income Domain numerator'].fillna(9, inplace=True)
inc_dep['inc_dep_indicator'] = inc_dep['Income Domain numerator'] / inc_dep['Total population: mid 2015 (excluding prisoners)']
inc_dep[['LSOA code (2011)', 'inc_dep_indicator']].to_csv(processed_data_directory + 'inc_dep_england.csv', index=False)

In [ ]:
# Wales
filepath = raw_data_directory + 'WIMD 2019 Indicator data by lower super output area.ods'
get_raw_data("https://statswales.gov.wales/Download/File?fileId=626", filepath)
xls = pd.ExcelFile(filepath)
wal_inc_dep = pd.read_excel(xls, sheet_name=2, skiprows=2)
wal_inc_dep = wal_inc_dep.loc[1:]
wal_inc_dep[['Lower Layer Super Output Area Code', 'People in Income Deprivation (%)']].to_csv(processed_data_directory + 'inc_dep_wal.csv', index=False)

In [ ]:
# Scotland
url = "https://www.gov.scot/binaries/content/documents/govscot/publications/statistics/2020/01/scottish-index-of-multiple-deprivation-2020-indicator-data/documents/simd_2020_indicators/simd_2020_indicators/govscot%3Adocument/SIMD%2B2020v2%2B-%2Bindicators.xlsx"
filepath=raw_data_directory + 'SIMD%2B2020v2%2B-%2Bindicators.xlsx'
get_raw_data(url, filepath)
xls = pd.ExcelFile(filepath)
scotland_inc_dep = pd.read_excel(xls, sheet_name=2)
scotland_inc_dep[['Data_Zone', 'Income_rate']].to_csv(processed_data_directory+'inc_dep_scotland.csv', index=False)

In [ ]:
# Northern Ireland
url = "https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/NIMDM17_SOAresults.xls"
filepath = raw_data_directory + 'NIMDM17_SOAresults.xls'
get_raw_data(url, filepath)
xls = pd.ExcelFile(filepath)
ni_inc_dep = pd.read_excel(xls, sheet_name=2)
ni_inc_dep[['SOA2001', "Proportion of the population living in households whose equivalised income is below 60 per cent of the NI median \n(%)"]].to_csv(processed_data_directory+'inc_dep_ni.csv', index=False)

### Proportion of population with no car access (UK Census 2011). E,S,W,NI.

In [ ]:
filepath = raw_data_directory + 'CARVAN_LSOADZ_England_Northern_Ireland_Scotland_Wales_Descriptions.csv'
url = "https://s3-eu-west-1.amazonaws.com/statistics.digitalresources.jisc.ac.uk/dkan/files/CARVAN/CARVAN_LSOADZ_England_Northern_Ireland_Scotland_Wales_Descriptions.csv"
get_raw_data(url, filepath)
car_df = pd.read_csv(filepath)
car_df = car_df[['GEO_CODE', 'Car or van availability : Total\ Car or van availability - Unit : Households', 'Car or van availability : No cars or vans in household - Unit : Households']]
car_df.columns = ['geo_code', 'households', 'no_cars_in_household']
car_df.to_csv(processed_data_directory + 'car_access.csv', index=False)

## Need for family food support

### Free school meal eligibility. E,S,W,NI.

- state-funded nursery schools
- state-funded primary schools
- state-funded secondary schools
- special schools, including state-funded special schools and non-maintained special schools
- pupil referral units, including free school alternative provision and academy alternative provision

https://explore-education-statistics.service.gov.uk/data-catalogue/free-school-meals-autumn-term/2020-21-autumn-term

In [ ]:
# Need some additional lookups to standardise the FSM data.

lookup_url = "https://www.arcgis.com/sharing/rest/content/items/e7824b1475604212a2325cd373946235/data"
lookup_filepath = raw_data_directory + 'PCD_OA_LSOA_MSOA_LAD_MAY22_UK_LU.zip'
uncompressed_filepath = raw_data_directory + 'fsm_lookup'
get_raw_data(lookup_url, lookup_filepath, compressed_file=True, uncompressed_filepath=uncompressed_filepath)
fsm_lookup = pd.read_csv(raw_data_directory + 'fsm_lookup/PCD_OA_LSOA_MSOA_LAD_MAY22_UK_LU.csv', encoding='latin-1')

utla_lookup = gpd.read_file("https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LTLA21_UTLA21_EW_LU_9bbac05558b74a88bda913ad5bf66917/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")

# There's a fair bit going on in the below, but, essentially, 
# - we take the UTLA17CD for Buckinghamshire and Northamptonshire, 
# - we take the LAD11CD for Gateshead and Northumberland
# and then add these to the LAD CD from the fsm_lookup derived above to 
# generate a bespoke lookup that fits onto the fsm data for England

fsm_lookup = fsm_lookup[['lsoa11cd', 'ladcd', 'ladnm']].drop_duplicates()
fsm_lookup_eng = fsm_lookup.loc[fsm_lookup['lsoa11cd'].str.startswith('E')==True,:]
fsm_lookup_eng = fsm_lookup_eng.merge(utla_lookup[['LTLA21CD', 'UTLA21CD']], left_on='ladcd', right_on='LTLA21CD', how='left', indicator=True)
fsm_lookup_eng.rename({'_merge':'merge1'}, inplace=True, axis=1)
bespoke_lookup = fsm_lookup_eng[['lsoa11cd', 'UTLA21CD']].drop_duplicates()

oldutla_lookup = gpd.read_file("https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LSOA11_UTLA17_EW_LU_fc16626637fa480f8164e11378f5a1e4/FeatureServer/0/query?outFields=*&where=UTLA17NM%20IN%20(%27Buckinghamshire%27,%20%27Northamptonshire%27)&f=geojson")
bespoke_lookup = bespoke_lookup.merge(oldutla_lookup[['LSOA11CD', 'UTLA17CD']], left_on='lsoa11cd', right_on='LSOA11CD', how='left')

lad2011lookup = gpd.read_file("https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LSOA01_LSOA11_LAD11_EW_LU_ddfe1cd1c2784c9b991cded95bc915a9/FeatureServer/0/query?outFields=*&where=LAD11NM%20IN%20(%27Gateshead%27,%20%27Northumberland%27)&f=geojson")
bespoke_lookup = bespoke_lookup.merge(lad2011lookup[['LSOA11CD', 'LAD11CD']], left_on='lsoa11cd', right_on='LSOA11CD', how='left')

bespoke_lookup['lookup_cd'] = bespoke_lookup['LAD11CD'].fillna(bespoke_lookup['UTLA17CD']).fillna(bespoke_lookup['UTLA21CD'])

In [ ]:
# England FSM data
url = 'https://content.explore-education-statistics.service.gov.uk/api/releases/df258c3a-5be2-4b6c-9f20-08d88fd210c7/files'
filepath = raw_data_directory + 'fsm_eng.zip'
uncompressed_filepath = raw_data_directory + 'fsm_eng'
get_raw_data(url, filepath, compressed_file=True, uncompressed_filepath=uncompressed_filepath)

fsm_eng = pd.read_csv(raw_data_directory + 'fsm_eng/data/fsm_new_starts_autumn20.csv')
fsm_eng = fsm_eng.loc[fsm_eng['time_period']==202021,:]
fsm_eng = fsm_eng[(fsm_eng['phase']=='Total') & (fsm_eng['geographic_level']=='Local authority')]

fsm_eng_lsoa = fsm_eng.merge(bespoke_lookup, right_on='lookup_cd', left_on ='new_la_code', how='right', indicator=True)
fsm_eng_lsoa.rename({'_merge': 'merge2'}, inplace=True, axis=1)

fsm_eng_lsoa[['lsoa11cd', 'fsm_eligible_percent']].to_csv(processed_data_directory + 'fsm_england.csv', index=False)

In [ ]:
# Wales
url = "https://www.gov.wales/sites/default/files/statistics-and-research/2022-08/schools-census-results-as-at-february-2022-tables-227.ods"
filepath = raw_data_directory + "schools-census-results-as-at-february-2022-tables-227.ods"
get_raw_data(url, filepath)
xls = pd.ExcelFile(filepath)
fsm_wal = pd.read_excel(xls, 13, skiprows=2)
worksheet_title = pd.read_excel(xls,13,nrows=1)

fsm_wal = fsm_wal[['Local authority', 'All pupils - total - per cent']].dropna()
fsm_wal = fsm_wal.replace('The Vale of Glamorgan', 'Vale of Glamorgan')
wal_test = fsm_wal.merge(fsm_lookup[fsm_lookup.lsoa11cd.str[0]=='W'], left_on='Local authority', right_on='ladnm', how='right', indicator=True)
wal_test.rename({'All pupils - total - per cent':'fsm_eligible_percent'}, inplace=True, axis=1)
wal_test[['lsoa11cd', 'fsm_eligible_percent']].to_csv(processed_data_directory + 'fsm_wales.csv', index=False)

In [ ]:
# Scotland
url = "https://www.gov.scot/binaries/content/documents/govscot/publications/statistics/2019/07/pupil-census-supplementary-tables/documents/pupil-census-2021-supplementary-statistics/pupil-census-2021-supplementary-statistics/govscot%3Adocument/Pupils%2BCensus%2BSupplementary%2BStatistics%2B2021%2BV3.xlsx"
filepath = raw_data_directory + 'Pupils+Census+Supplementary+Statistics+2021+V3.xlsx'
get_raw_data(url, filepath)

xls = pd.ExcelFile(raw_data_directory + 'Pupils+Census+Supplementary+Statistics+2021+V3.xlsx')
fsm_scot_primary = pd.read_excel(xls, sheet_name='Table 6.17', header=[0,1], skiprows=1)
fsm_scot_primary.columns = ['LAD', '2018_count', '2018_percent', '2019_count', '2019_percent', '2020_count', '2020_percent', '2021_count', '2021_percent', 'blank', 'blank']
fsm_scot_primary.replace({"\(3\)":""}, regex=True, inplace=True)

fsm_scot_secondary = pd.read_excel(xls, sheet_name='Table 7.14', header=[0,1], skiprows=1)
fsm_scot_secondary.columns = ['LAD', '2018_count', '2018_percent', '2019_count', '2019_percent', '2020_count', '2020_percent', '2021_count', '2021_percent', 'blank', 'blank', 'blank']

fsm_scot = fsm_scot_primary.merge(fsm_scot_secondary, on='LAD', how='outer', suffixes=('_primary', '_secondary'))
fsm_scot = fsm_scot[['LAD', '2021_count_primary', '2021_percent_primary', '2021_count_secondary', '2021_percent_secondary']].dropna()

fsm_scot['2021_primary_total_population'] = 100 * fsm_scot['2021_count_primary'] / fsm_scot['2021_percent_primary']
fsm_scot['2021_secondary_total_population'] = 100 * fsm_scot['2021_count_secondary'] / fsm_scot['2021_percent_secondary']
fsm_scot['fsm_percent'] = 100 * (fsm_scot['2021_count_primary'] + fsm_scot['2021_count_secondary']) / (fsm_scot['2021_primary_total_population'] + fsm_scot['2021_secondary_total_population'])

fsm_scot_lsoa = fsm_scot[['LAD', 'fsm_percent']].merge(fsm_lookup[fsm_lookup['lsoa11cd'].str[0]=='S'], left_on='LAD', right_on='ladnm', how='right', indicator=True)
fsm_scot_lsoa[['lsoa11cd', 'fsm_percent']].to_csv(processed_data_directory + 'fsm_scotland.csv', index=False)


In [ ]:
#NI FSM
ni_lookup = pd.DataFrame()
for lad in ['N09000001', 'N09000002', 'N09000003', 'N09000004', 'N09000005',
       'N09000006', 'N09000007', 'N09000008', 'N09000009', 'N09000010',
       'N09000011']:
    ni_lookup_by_lad = gpd.read_file("https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/SA11_LSOA11_LAD18_NI_LU_classificationv2_942e0f54c76a4998becb14986dc0b0ca/FeatureServer/0/query?outFields=LSOA11CD,LSOA11NM,LAD18CD,LAD18NM&where=LAD18CD='%s'&f=geojson" % lad)
    ni_lookup = pd.concat([ni_lookup, ni_lookup_by_lad[['LSOA11CD','LSOA11NM','LAD18CD','LAD18NM']]])
ni_lookup = ni_lookup.reset_index()

url = "https://www.ninis2.nisra.gov.uk/Download/Children%20Education%20and%20Skills/School%20Leavers%20Free%20School%20Meal%20Entitlement%20(administrative%20geographies).ods"
filepath = raw_data_directory + 'School%20Leavers%20Free%20School%20Meal%20Entitlement%20(administrative%20geographies).ods'
get_raw_data(url, filepath)
xls = pd.ExcelFile(filepath)
fsm_ni = pd.read_excel(xls, sheet_name='LGD2014', skiprows=3)
fsm_ni = fsm_ni.dropna()
fsm_ni['LGD2014'] = fsm_ni['LGD2014'].str.replace('Ards and North Down', 'North Down and Ards')
fsm_ni['LGD2014'] = fsm_ni['LGD2014'].str.replace(' City', '')
fsm_ni = fsm_ni.merge(ni_lookup, how='right', left_on='LGD2014', right_on='LAD18NM')
fsm_ni = fsm_ni[['LSOA11CD', 'Non-FSME School Leavers', 'FSME School Leavers']].groupby('LSOA11CD').first()
fsm_ni['prop_FSME_school_leavers'] = fsm_ni['FSME School Leavers'] / (fsm_ni['FSME School Leavers'] + fsm_ni['Non-FSME School Leavers'])
fsm_ni.to_csv(processed_data_directory + 'fsm_ni.csv')

### Healthy start voucher usage (England and Wales only). E,W.

In [ ]:
eng_url = 'https://www.healthystart.nhs.uk/wp-content/uploads/2023/02/England-Uptake-Data.xlsx'
eng_filepath = raw_data_directory + 'England-Uptake-Data.xlsx'
get_raw_data(eng_url, eng_filepath)

wal_url = 'https://www.healthystart.nhs.uk/wp-content/uploads/2023/02/Wales-Uptake-Data.xlsx'
wal_filepath = raw_data_directory + 'Wales-Uptake-Data.xlsx'
get_raw_data(wal_url, wal_filepath)

census_url = 'https://s3-eu-west-1.amazonaws.com/statistics.digitalresources.jisc.ac.uk/dkan/files/2021/ONS/release1/census-2021-first-results-england-wales.xlsx'
census_filepath = raw_data_directory + 'census-2021-first-results-england-wales.xlsx'
get_raw_data(census_url, census_filepath)

xls_eng = pd.ExcelFile(raw_data_directory + "England-Uptake-Data.xlsx")
eng_hsv = pd.read_excel(xls_eng, sheet_name='Feb-23', skiprows=7)

xls_wal = pd.ExcelFile(raw_data_directory + "Wales-Uptake-Data.xlsx")
wal_hsv = pd.read_excel(xls_wal, sheet_name='Feb-23',skiprows=4)

xls_pop = pd.ExcelFile(raw_data_directory + 'census-2021-first-results-england-wales.xlsx')
pop_census_21 = pd.read_excel(xls_pop, sheet_name='P02', skiprows=7)

HS_uptake = pd.concat([eng_hsv, wal_hsv])

# Local Authority level. Some data cleaning required:

# Remove 'Teaching' from several local authority names in Wales
HS_uptake['Local Authority'] = HS_uptake['Local Authority'].astype(str).str.replace(' Teaching',"")
# Change local authority wording to match lookup 
HS_uptake['Local Authority'] = HS_uptake['Local Authority'].astype(str).str.replace("Taff","Taf")
HS_uptake['Local Authority'] = HS_uptake['Local Authority'].astype(str).str.replace('Kings Lynn and West Norfolk',"King's Lynn and West Norfolk")

lookup = pd.read_csv(raw_data_directory + 'fsm_lookup/PCD_OA_LSOA_MSOA_LAD_MAY22_UK_LU.csv', encoding='Latin-1')

# join look up table to census poualtion (2021) for all and child popualtions
lookup =lookup.merge(pop_census_21[['Area code [note 2]', 'All persons', 'Aged 4 years and under\n[note 12]']], left_on='ladcd', right_on='Area code [note 2]', how='left')

# join healthy start to lcoal authority in look up table
HS_uptake =lookup.merge(HS_uptake, left_on='ladnm', right_on ='Local Authority', how='left', indicator=True)

HS_uptake_lsoa = HS_uptake.groupby('lsoa11cd').agg({'ladnm':'first',
                                                    'Country':'first',
                                                    'Local Authority':'first',
                                                    'Total Entitled Beneficiaries':'first',
                                                    'Total Eligible Beneficiaries':'first',
                                                    'All persons':'first', 
                                                    'Uptake (%)':'first', 
                                                    'Aged 4 years and under\n[note 12]':'first',
                                                    '_merge':'first'})

HS_uptake_lsoa['Percentage_Population_Eligible_HS'] =HS_uptake_lsoa['Total Eligible Beneficiaries']/HS_uptake_lsoa['All persons']*100
HS_uptake_lsoa['Percentage_under_5_Population_Eligible_HS'] =HS_uptake_lsoa['Total Eligible Beneficiaries']/HS_uptake_lsoa['Aged 4 years and under\n[note 12]']*100
HS_uptake_lsoa['Percentage_Population_Using_HS'] =HS_uptake_lsoa['Total Entitled Beneficiaries']/HS_uptake_lsoa['All persons']*100
HS_uptake_lsoa['Percentage_under_5_Population_Using_HS'] =HS_uptake_lsoa['Total Entitled Beneficiaries']/HS_uptake_lsoa['Aged 4 years and under\n[note 12]']*100
HS_uptake_lsoa.to_csv(processed_data_directory + 'HS_uptake_lsoa.csv')

### Distance to nearest food bank (https://www.givefood.org.uk/, accessed 2022-08-19). E,S,W,NI. 

In [ ]:

if not os.path.exists(raw_data_directory + 'givefood.csv'):
    x = requests.get('https://www.givefood.org.uk/api/2/foodbanks/', headers={'x-api-version':'2'})
    foodbanks = json.loads(x.content)
    foodbanks_df = pd.json_normalize(foodbanks)
    foodbanks_df.to_csv(raw_data_directory + 'givefood.csv')
else:
    foodbanks_df = pd.read_csv(raw_data_directory + 'givefood.csv')
    
foodbanks_df['lat'] = pd.to_numeric(foodbanks_df.loc[:, 'lat_lng'].str.split(',').apply(operator.itemgetter(0)))
foodbanks_df['lng'] = pd.to_numeric(foodbanks_df.loc[:, 'lat_lng'].str.split(',').apply(operator.itemgetter(1)))
foodbanks_df = gpd.GeoDataFrame(foodbanks_df, 
                                geometry=gpd.points_from_xy(foodbanks_df['lng'], foodbanks_df['lat']),
                                crs='EPSG:4326')

pntsB = foodbanks_df[['slug', 'geometry']]
pntsB.set_index('slug', inplace=True)

pntsA = postcodes[['PCD', 'geometry']]
pntsA.set_index('PCD', inplace=True)

Polygon = country_outline.envelope.values[0]

dist = calculate_bipartite_nearest_distance(pntsA, pntsB, Polygon)
dist.to_csv(processed_data_directory + 'postcode_to_nearest_foodbank_distance.csv', index=True)

## Fuel Poverty

### Proportion of households in fuel poverty (2017 - 2020). E,S,W.

Low Income Low Energy Efficiency (LILEE) fuel poverty metric (2020 data) - England https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1056802/fuel-poverty-methodology-handbook-2022-lilee-with-projection.pdf

https://www.gov.uk/government/statistics/sub-regional-fuel-poverty-data-2022


Fuel poverty calculation (England)- estimates the number and proportion of fuel poor households for smaller geographical areas and measured using proxy measures available for low level geographic areas and constrained to the national data drawn from the English Housing Survey (EHS)


Application of the sub-regional model requires specific local area data for a variety of demographic and socio-economic factors, which are derived from the 2011 Census small area datasets

Ideally, the EHS data would be used directly to model at the local level. Unfortunately, this is not possible because of the relatively small sample used for the survey, which does not give sufficient coverage for each of the 309 Local Authorities in England or the individual Census Output Areas (COAs) 

To allow results to be produced at this level, we use the EHS data to create a model which can then be applied to a national (household level) dataset. The limitation of this is the possibility that small areas which are atypical in condition are unlikely to be identified by the model. It is therefore essential, wherever possible, to compare the modelled results to local data.


Model:
1. Regression model testing: 
- A large number of independent variables are individually tested against the binary fuel poverty indicator to determine which have the highest coefficient of determination, i.e. which are most sensitive in predicting the fuel poverty outcome.
- Variables are then grouped according to this coefficient and an iterative selection is made by running logistic regression on a developing group of these variables, with those least likely to contribute to final model predictions being dropped at each stage
- The final set of variables contributing to the model varies each year depending on the results of the regression testing.


The 2020 LILEE sub-regional fuel poverty model is based on the following seven variables, derived from public and commercial sources:
- Tenure – owner occupied, private rented, social rented (Experian)
- Dwelling age – pre-1919, 1919 to 1944, 1945 to 1980, post-1980 (Experian)
- Household composition (Experian)
- Mosaic household classifications by postcode – 15 socio-economic groups as detailed in the Mosaic guide (Experian)
- Dwelling type – terraced, detached, semi-detached, bungalow, flat (Experian)
- Government office region (Experian)
- Employment status - proportion of households in COA with household member in full time employment (2011 Census)

2. Model application
- The coefficients for each category of the final model’s independent variables are run to give the predicted outcome for each household in each area. 
- These are then combined to produce the total numbers and proportion of fuel poor at COA level.

3. Out put area cloning (addressing missing data):
- After this process, a small number of COAs have missing fuel poverty data. 
- These gaps are filled by identifying COAs with complete data, which share identical or very similar characteristics to the missing cases. 
- Census data is used for this comparison as it records proportions of particular characteristic rather than discrete categories, and so the process can more accurately match pairs of COAs. The Census variables used in this matching process were those found in the final regression model, or which were omitted at a very late stage of testing. 

4. Consolidation to national figures:
- In order to provide an exact match between the modelled household totals and the national figures (for all households and fuel poor households), a final adjustment is made in which small incremental changes are made randomly at a COA level, until the totals of all households and fuel poor households match the published regional totals

5. The adjusted figures are re-checked against the published totals before the data is aggregated to LSOA, Local Authority, Parliamentary Constituency and Region level.


Welsh equivalent of fuel poverty data: https://gov.wales/sites/default/files/statistics-and-research/2020-03/welsh-housing-conditions-survey-whcs-2017-18-local-area-fuel-poverty-estimates-modelling-and-results-summary-071.pdf (taken from pdf)


Scottish equivalent of fuel poverty data: https://statistics.gov.scot/resource?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Ffuel-poverty-shcs (local authority not data zone) 
A household is considered to be in fuel poverty if, in order to maintain a satisfactory heating regime:
the total fuel costs necessary for the home are more than 10% of the household’s adjusted net income (after housing costs), and
if after deducting fuel costs, benefits received for a care need or disability and childcare costs, the household’s remaining adjusted net income is insufficient to maintain an acceptable standard of living. The remaining adjusted net income must be at least 90% of the UK Minimum Income Standard to be considered an acceptable standard of living, with an additional amount added for households in remote rural, remote small town and island areas.
Extreme fuel poverty follows the same definition except that a household would have to spend more than 20% of its adjusted net income (after housing costs) on total fuel costs and maintain a satisfactory heating regime.

In [ ]:
# England URL: 

url = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1081193/sub-regional-fuel-poverty-2022-tables-ods.ods'
filepath = raw_data_directory + 'sub-regional-fuel-poverty-2022-tables-ods.ods'
get_raw_data(url, filepath)
xls_eng = pd.ExcelFile(raw_data_directory + "sub-regional-fuel-poverty-2022-tables-ods.ods")
fuel_eng = pd.read_excel(xls_eng, sheet_name='Table_3', skiprows=2)

# Wales PDF:
# https://gov.wales/sites/default/files/statistics-and-research/2020-03/welsh-housing-conditions-survey-whcs-2017-18-local-area-fuel-poverty-estimates-modelling-and-results-summary-071.pdf
wales_data="""
Local_authority,Households in fuel poverty,Percentage of households in fuel poverty 
Isle of Anglesey,5000,17
Gwynedd,12000,23
Conwy,6000,12
Denbighshire,5000,12
Flintshire,6000,9
Wrexham,6000,10
Ceredigion,7000,21
Pembrokeshire,8000,15
Carmarthenshire,11000,14
Swansea,11000,10
Neath Port Talbot,7000,11
Bridgend,5000,8
The Vale of Glamorgan,5000,8
Cardiff,14000,9
Rhondda Cynon Taff,12000,11
Caerphilly,7000,9
Blaenau Gwent,3000,11
Torfaen,3000,8
Monmouthshire,4000,10
Newport,6000,9
Powys,10000,17
Merthyr Tydfil,3000,10"""

fuel_wal = pd.read_csv(io.StringIO(wales_data))

fuel_scot = pd.read_csv('https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Ffuel-poverty-shcs')
fuel_scot = fuel_scot[(fuel_scot['FeatureType']=='Council Area') & \
                      (fuel_scot['DateCode']=='2017-2019') & \
                      (fuel_scot['Units']=='Percent of households in fuel poverty') & \
                      (fuel_scot['Age of Dwelling']=='All') & \
                      (fuel_scot['Type Of Dwelling']=='All') & \
                      (fuel_scot['Number of Bedrooms']=='All') & \
                      (fuel_scot['Type Of Tenure']=='All') & \
                      (fuel_scot['Household Type']=='All')]

lookup = pd.read_csv(raw_data_directory + 'fsm_lookup/PCD_OA_LSOA_MSOA_LAD_MAY22_UK_LU.csv', encoding='Latin-1', usecols=['lsoa11cd', 'ladcd', 'ladnm'])
lookup = lookup.drop_duplicates()

xls_pop = pd.ExcelFile(raw_data_directory + 'census-2021-first-results-england-wales.xlsx')
pop_census_21 = pd.read_excel(xls_pop, sheet_name='P02', skiprows=7)

lookup =lookup.merge(pop_census_21[['Area code [note 2]', 'All persons', 'Aged 4 years and under\n[note 12]']], left_on='ladcd', right_on='Area code [note 2]', how='left')

fuel_wal['Local_authority'] =fuel_wal['Local_authority'].astype(str).str.replace("Taff","Taf")
fuel_wal['Local_authority'] =fuel_wal['Local_authority'].astype(str).str.replace("The Vale","Vale")

# Join to look-up 
fuel_wal = lookup.merge(fuel_wal, left_on='ladnm', right_on ='Local_authority', how='left')

# Remove areas with no match i.e. not in Wales
fuel_wal=fuel_wal.dropna(subset=['Percentage of households in fuel poverty '])

fuel_wales_to_join =fuel_wal[['lsoa11cd','Percentage of households in fuel poverty ']]
# Rename for consistent formatting
fuel_wales_to_join.columns =['lsoa11cd','Percent of households in fuel poverty']

# Join to look-up 
fuel_scot =lookup.merge(fuel_scot, left_on='ladcd', right_on ='FeatureCode', how='left')

# Remove areas with no match i.e. not in Scotland
fuel_scot =fuel_scot.dropna(subset=['Value'])

# Select columns to join to wider fuel poverty df
fuel_scot_to_join =fuel_scot[['lsoa11cd','Value']]
fuel_scot_to_join.columns =['lsoa11cd','Percent of households in fuel poverty']

# Select columns to join to wider fuel poverty df
fuel_eng_to_join = fuel_eng[['LSOA Code','Proportion of households fuel poor (%)']]
# Rename for consistent formatting 
fuel_eng_to_join.columns =['lsoa11cd','Percent of households in fuel poverty']

fuel =pd.concat([fuel_scot_to_join,fuel_wales_to_join,fuel_eng_to_join])

fuel = fuel.drop_duplicates()
fuel.dropna(inplace=True)
fuel.to_csv(processed_data_directory + 'fuel_poverty.csv', index=False)

### Prepayment meter prevalence, 2017. E,S,W.

https://www.gov.uk/government/statistics/electric-prepayment-meter-statistics

In [ ]:
url = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/789328/LSOA-prepayment-electricity-2017.csv'
filepath = raw_data_directory + 'LSOA-prepayment-electricity-2017.csv'
get_raw_data(url, filepath)

postcode_url = 'https://s3-eu-west-1.amazonaws.com/statistics.digitalresources.jisc.ac.uk/dkan/files/Postcode_Counts_and_Deprivation_Ranks/postcodes.zip'
postcode_filepath = raw_data_directory + 'postcodes.zip'
uncompressed_filepath = raw_data_directory + 'postcodes'
get_raw_data(postcode_url, postcode_filepath, compressed_file=True, uncompressed_filepath=uncompressed_filepath)

prepayment = pd.read_csv(raw_data_directory + 'LSOA-prepayment-electricity-2017.csv', skiprows=1)
households_lookup = pd.read_csv(raw_data_directory + 'postcodes/postcodes_data.csv', usecols=['PCD', 'Occupied_Households','LSOA_DZ_Code'])
households_lookup = households_lookup.groupby('LSOA_DZ_Code')['Occupied_Households'].sum()
prepayment = prepayment.merge(households_lookup, left_on='Lower Layer Super Output Area (LSOA) Code', right_index=True, how='left', indicator=True)

prepayment.to_csv(processed_data_directory + 'prepayment_meters.csv', index=False)